In [ ]:
import pyodbc
from meyerDB import cable_connection
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML, display
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeavePOut
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn import linear_model
from sklearn.neighbors import KNeighborsRegressor
import sklearn.preprocessing as pp

# init db connection
conn = pyodbc.connect(cable_connection)
cursor = conn.cursor()
print('Database connection ok')

def display_table(data):
    html = "<table>"
    for row in data:
        html += "<tr>"
        for field in row:
            try:
                value = str(round(100*field, 1)).replace('.', ',') + '%'
            except:
                value = field
            html += "<td><h4>%s</h4><td>"%(value)
        html += "</tr>"
    html += "</table>"
    display(HTML(html))

In [ ]:
# Whole ship cabling readiness
# Predictors:
# GT
# Algorithms:
# Mean

# Get the data

#get ship data
cursor.execute("SELECT project_id, sum(amount) FROM routed GROUP BY project_id")
data = np.array(cursor.fetchall()).astype('int32')
ships = data[:, 0]

# init lpo split
p = 2 #ships
lpo = LeavePOut(p)

# train and evaluate models with lpo
table_rows = [['Model', 'mean', 'std', 'max', 'min']]
scores = []
for train_i, test_i in lpo.split(ships):
    s1 = ships[test_i[0]]
    s2 = ships[test_i[1]]

    cursor.execute(
        "SELECT "
        "(SELECT avg(cable_count) as cables_per_week GROUP BY rw FROM("
        " SELECT rw, project_id, sum(amount) as cable_count FROM ship_readiness WHERE rw IS NOT NULL GROUP BY project_id, rw) cable_counts"
        " WHERE cable_counts.project_id <> {} AND cable_counts.project_id <> {}) predictions"
        )


    score = mape(y_test, preds)
    scores.append(score)

scores = np.array(scores)
table_rows.append((model_name, 1-scores.mean(), scores.std(), 1-scores.min(), 1-scores.max()))
display_table(table_rows)